In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_synthetic_fixed_cost_runs(acq, tfn):
    runs = api.runs(path="ziv-scully-group/Gittins for Bayesian Optimization", filters={
        "sweep": "dksankho", 
        "config.problem": tfn,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
    'ExpectedImprovement':'ExpectedImprovement',
    'ThompsonSampling':'ThompsonSampling', 
    'KnowledgeGradient':'KnowledgeGradient',
    'MultiStepLookaheadEI':'ExpectedImprovement_MultiStep',
    'Gittins_Lambda_01':'Gittins_Lambda01',
    'Gittins_Lambda_001':'Gittins_Lambda001',
    'Gittins_Lambda_0001':'Gittins_Lambda0001',
    'Gittins_Step_Divide2':'Gittins_Step_Divide2',
    'Gittins_Step_Divide5':'Gittins_Step_Divide5',
    'Gittins_Step_Divide10':'Gittins_Step_Divide10',
    'Gittins_Step_EIpu':'Gittins_Step_EIPC',
    }
target_functions_with_optima = {
    'NN': 0.08956228956228955
  }

In [3]:
grouped_runs = {(a,t): load_synthetic_fixed_cost_runs(a,t) for a in acquisition_functions.keys() for t in target_functions_with_optima.keys()}

100%|███████████████████████████████████████████| 18/18 [00:21<00:00,  1.19s/it]


In [ ]:
grouped_runs[('ExpectedImprovement','NN')]

In [ ]:
for a in acquisition_functions.keys():
    for (t,opt) in target_functions_with_optima.items():
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed]).T
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed]).T
        
        regret_per_seed = best_observed_per_seed - opt

        print("regret_per_seed:", regret_per_seed)

        regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
        regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
        regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)

        np.savetxt(f"empirical/Empirical_{t}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')